## Imports

In [ ]:
# pip install --upgrade scikit-learn

In [48]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [49]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../Datasets/Tabular/titanic/"

## Load data

In [50]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [51]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [52]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [53]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)
get_Title_from_Name = None

# YOUR CODE HERE
get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [54]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [55]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [56]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df['Title'] = df.Title.map(title_dictionary)
df_test['Title'] = df_test.Title.map(title_dictionary)

In [57]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [13]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [14]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [15]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [64]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
""";

# YOUR CODE HERE
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('ordinal', preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=nan))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [65]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [66]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [67]:
"""
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":
  "Random Forest":
  "AdaBoost":
  "Skl GBM":
  "Skl HistGBM":
  "XGBoost":
  "LightGBM":
  "CatBoost":
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]
""";


# YOUR CODE HERE
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(random_state=0),
  "Extra Trees":ExtraTreesClassifier(random_state=0),
  "Random Forest":RandomForestClassifier(random_state=0),
  "AdaBoost":AdaBoostClassifier(random_state=0),
  "Skl GBM":GradientBoostingClassifier(random_state=0),
  "Skl HistGBM":GradientBoostingClassifier(random_state=0),
  "XGBoost":XGBClassifier(random_state=0),
  "LightGBM":LGBMClassifier(random_state=0),
  "CatBoost":CatBoostClassifier(random_state=0)
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier(random_state=0))])

In [68]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [69]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
# YOUR CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,
    random_state=4)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# YOUR CODE HERE
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    model.fit(x_train,y_train)
    total_time = time.time() - start_time
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6896184	total: 1.9ms	remaining: 1.89s
1:	learn: 0.6858162	total: 3.66ms	remaining: 1.82s
2:	learn: 0.6796524	total: 6.88ms	remaining: 2.28s
3:	learn: 0.6738140	total: 9.4ms	remaining: 2.34s
4:	learn: 0.6690768	total: 11.3ms	remaining: 2.24s
5:	learn: 0.6636089	total: 14.4ms	remaining: 2.38s
6:	learn: 0.6605114	total: 17.9ms	remaining: 2.54s
7:	learn: 0.6558070	total: 19.9ms	remaining: 2.46s
8:	learn: 0.6504937	total: 22.7ms	remaining: 2.5s
9:	learn: 0.6449112	total: 27.5ms	remaining: 2.72s
10:	learn: 0.6396662	total: 31.3ms	remaining: 2.81s
11:	learn: 0.6347641	total: 34.2ms	remaining: 2.81s
12:	learn: 0.6297268	total: 36.7ms	remaining: 2.78s
13:	learn: 0.6246363	total: 38.9ms	remaining: 2.74s
14:

173:	learn: 0.3957136	total: 528ms	remaining: 2.51s
174:	learn: 0.3954766	total: 531ms	remaining: 2.5s
175:	learn: 0.3950583	total: 533ms	remaining: 2.49s
176:	learn: 0.3947632	total: 535ms	remaining: 2.48s
177:	learn: 0.3943714	total: 536ms	remaining: 2.48s
178:	learn: 0.3938671	total: 538ms	remaining: 2.47s
179:	learn: 0.3934803	total: 541ms	remaining: 2.46s
180:	learn: 0.3928976	total: 543ms	remaining: 2.46s
181:	learn: 0.3925546	total: 545ms	remaining: 2.45s
182:	learn: 0.3920779	total: 547ms	remaining: 2.44s
183:	learn: 0.3916052	total: 548ms	remaining: 2.43s
184:	learn: 0.3912415	total: 550ms	remaining: 2.42s
185:	learn: 0.3910881	total: 552ms	remaining: 2.41s
186:	learn: 0.3908220	total: 554ms	remaining: 2.41s
187:	learn: 0.3905707	total: 555ms	remaining: 2.4s
188:	learn: 0.3903207	total: 557ms	remaining: 2.39s
189:	learn: 0.3899870	total: 560ms	remaining: 2.39s
190:	learn: 0.3897249	total: 562ms	remaining: 2.38s
191:	learn: 0.3895524	total: 570ms	remaining: 2.4s
192:	learn: 0.3

353:	learn: 0.3551402	total: 1.07s	remaining: 1.96s
354:	learn: 0.3548976	total: 1.08s	remaining: 1.96s
355:	learn: 0.3547248	total: 1.08s	remaining: 1.96s
356:	learn: 0.3545924	total: 1.1s	remaining: 1.98s
357:	learn: 0.3543355	total: 1.1s	remaining: 1.98s
358:	learn: 0.3541753	total: 1.11s	remaining: 1.99s
359:	learn: 0.3538580	total: 1.13s	remaining: 2.02s
360:	learn: 0.3537035	total: 1.14s	remaining: 2.01s
361:	learn: 0.3535298	total: 1.14s	remaining: 2.01s
362:	learn: 0.3534548	total: 1.14s	remaining: 2s
363:	learn: 0.3531389	total: 1.15s	remaining: 2s
364:	learn: 0.3528861	total: 1.15s	remaining: 2.01s
365:	learn: 0.3527405	total: 1.16s	remaining: 2.01s
366:	learn: 0.3526120	total: 1.16s	remaining: 2s
367:	learn: 0.3526025	total: 1.16s	remaining: 2s
368:	learn: 0.3525982	total: 1.16s	remaining: 1.99s
369:	learn: 0.3524225	total: 1.17s	remaining: 1.99s
370:	learn: 0.3523333	total: 1.17s	remaining: 1.99s
371:	learn: 0.3522013	total: 1.18s	remaining: 1.99s
372:	learn: 0.3519483	tota

569:	learn: 0.3268499	total: 1.8s	remaining: 1.35s
570:	learn: 0.3267761	total: 1.8s	remaining: 1.35s
571:	learn: 0.3266078	total: 1.8s	remaining: 1.35s
572:	learn: 0.3265273	total: 1.81s	remaining: 1.35s
573:	learn: 0.3264861	total: 1.81s	remaining: 1.35s
574:	learn: 0.3262212	total: 1.82s	remaining: 1.34s
575:	learn: 0.3261319	total: 1.82s	remaining: 1.34s
576:	learn: 0.3259612	total: 1.82s	remaining: 1.33s
577:	learn: 0.3257912	total: 1.82s	remaining: 1.33s
578:	learn: 0.3255936	total: 1.83s	remaining: 1.33s
579:	learn: 0.3254161	total: 1.83s	remaining: 1.33s
580:	learn: 0.3252433	total: 1.83s	remaining: 1.32s
581:	learn: 0.3251855	total: 1.84s	remaining: 1.32s
582:	learn: 0.3251131	total: 1.84s	remaining: 1.32s
583:	learn: 0.3250835	total: 1.84s	remaining: 1.31s
584:	learn: 0.3249539	total: 1.85s	remaining: 1.31s
585:	learn: 0.3247779	total: 1.85s	remaining: 1.31s
586:	learn: 0.3245304	total: 1.87s	remaining: 1.31s
587:	learn: 0.3244940	total: 1.87s	remaining: 1.31s
588:	learn: 0.3

736:	learn: 0.3076996	total: 2.31s	remaining: 825ms
737:	learn: 0.3075341	total: 2.32s	remaining: 824ms
738:	learn: 0.3075126	total: 2.32s	remaining: 821ms
739:	learn: 0.3073059	total: 2.33s	remaining: 817ms
740:	learn: 0.3072436	total: 2.33s	remaining: 814ms
741:	learn: 0.3072138	total: 2.33s	remaining: 810ms
742:	learn: 0.3071477	total: 2.33s	remaining: 807ms
743:	learn: 0.3070692	total: 2.33s	remaining: 803ms
744:	learn: 0.3069459	total: 2.34s	remaining: 800ms
745:	learn: 0.3067749	total: 2.35s	remaining: 800ms
746:	learn: 0.3065931	total: 2.35s	remaining: 797ms
747:	learn: 0.3063964	total: 2.35s	remaining: 794ms
748:	learn: 0.3061058	total: 2.36s	remaining: 790ms
749:	learn: 0.3059815	total: 2.36s	remaining: 787ms
750:	learn: 0.3058799	total: 2.36s	remaining: 783ms
751:	learn: 0.3058003	total: 2.36s	remaining: 780ms
752:	learn: 0.3057390	total: 2.37s	remaining: 776ms
753:	learn: 0.3056552	total: 2.37s	remaining: 773ms
754:	learn: 0.3055545	total: 2.37s	remaining: 770ms
755:	learn: 

939:	learn: 0.2839585	total: 3.01s	remaining: 192ms
940:	learn: 0.2838332	total: 3.01s	remaining: 189ms
941:	learn: 0.2836742	total: 3.02s	remaining: 186ms
942:	learn: 0.2836285	total: 3.03s	remaining: 183ms
943:	learn: 0.2835652	total: 3.03s	remaining: 180ms
944:	learn: 0.2834194	total: 3.04s	remaining: 177ms
945:	learn: 0.2831765	total: 3.04s	remaining: 173ms
946:	learn: 0.2830476	total: 3.04s	remaining: 170ms
947:	learn: 0.2830006	total: 3.04s	remaining: 167ms
948:	learn: 0.2829010	total: 3.05s	remaining: 164ms
949:	learn: 0.2828135	total: 3.06s	remaining: 161ms
950:	learn: 0.2826203	total: 3.06s	remaining: 158ms
951:	learn: 0.2823916	total: 3.06s	remaining: 154ms
952:	learn: 0.2823295	total: 3.06s	remaining: 151ms
953:	learn: 0.2822623	total: 3.07s	remaining: 148ms
954:	learn: 0.2822462	total: 3.07s	remaining: 145ms
955:	learn: 0.2820074	total: 3.07s	remaining: 141ms
956:	learn: 0.2818804	total: 3.08s	remaining: 138ms
957:	learn: 0.2816037	total: 3.08s	remaining: 135ms
958:	learn: 

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,87.150838,85.494071,3.871003
2,Skl GBM,86.592179,85.039526,0.289504
3,Skl HistGBM,86.592179,85.039526,0.278848
4,AdaBoost,84.916201,83.945982,0.363822
5,XGBoost,83.798883,82.766798,0.356335
6,LightGBM,83.798883,82.766798,0.269042
7,Random Forest,82.681564,81.047431,0.541025
8,Extra Trees,80.446927,79.229249,0.431549
9,Decision Tree,75.977654,75.052701,0.065197


In [70]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [71]:
"""
skf = model_selection.StratifiedKFold(
    # CODE HERE
)
"""
# YOUR CODE HERE
skf = model_selection.StratifiedKFold(
    n_splits = 10,
    shuffle=True,
    random_state=0
)
    


results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# YOUR CODE HERE
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    model.fit(x_train,y_train)
    total_time = time.time() - start_time
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = model_selection.cross_val_predict(model, x, y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:30:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:30:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

386:	learn: 0.3394307	total: 1.41s	remaining: 2.24s
387:	learn: 0.3392399	total: 1.42s	remaining: 2.23s
388:	learn: 0.3390058	total: 1.42s	remaining: 2.23s
389:	learn: 0.3389579	total: 1.42s	remaining: 2.23s
390:	learn: 0.3388748	total: 1.43s	remaining: 2.22s
391:	learn: 0.3386713	total: 1.43s	remaining: 2.22s
392:	learn: 0.3384332	total: 1.43s	remaining: 2.21s
393:	learn: 0.3384033	total: 1.44s	remaining: 2.21s
394:	learn: 0.3383695	total: 1.44s	remaining: 2.2s
395:	learn: 0.3379611	total: 1.44s	remaining: 2.2s
396:	learn: 0.3377554	total: 1.45s	remaining: 2.2s
397:	learn: 0.3376570	total: 1.45s	remaining: 2.2s
398:	learn: 0.3375360	total: 1.46s	remaining: 2.19s
399:	learn: 0.3374832	total: 1.46s	remaining: 2.19s
400:	learn: 0.3373226	total: 1.46s	remaining: 2.18s
401:	learn: 0.3373041	total: 1.46s	remaining: 2.18s
402:	learn: 0.3371820	total: 1.47s	remaining: 2.17s
403:	learn: 0.3370236	total: 1.47s	remaining: 2.17s
404:	learn: 0.3369558	total: 1.47s	remaining: 2.16s
405:	learn: 0.33

575:	learn: 0.3183456	total: 2.11s	remaining: 1.56s
576:	learn: 0.3182381	total: 2.12s	remaining: 1.55s
577:	learn: 0.3181404	total: 2.12s	remaining: 1.55s
578:	learn: 0.3180242	total: 2.12s	remaining: 1.54s
579:	learn: 0.3179276	total: 2.13s	remaining: 1.54s
580:	learn: 0.3179189	total: 2.13s	remaining: 1.54s
581:	learn: 0.3176342	total: 2.14s	remaining: 1.53s
582:	learn: 0.3174963	total: 2.14s	remaining: 1.53s
583:	learn: 0.3173570	total: 2.14s	remaining: 1.53s
584:	learn: 0.3172925	total: 2.15s	remaining: 1.52s
585:	learn: 0.3172781	total: 2.15s	remaining: 1.52s
586:	learn: 0.3171414	total: 2.15s	remaining: 1.51s
587:	learn: 0.3168967	total: 2.15s	remaining: 1.51s
588:	learn: 0.3168373	total: 2.16s	remaining: 1.51s
589:	learn: 0.3168206	total: 2.16s	remaining: 1.5s
590:	learn: 0.3166962	total: 2.16s	remaining: 1.5s
591:	learn: 0.3166776	total: 2.17s	remaining: 1.49s
592:	learn: 0.3166301	total: 2.17s	remaining: 1.49s
593:	learn: 0.3164740	total: 2.17s	remaining: 1.48s
594:	learn: 0.

769:	learn: 0.2987504	total: 2.84s	remaining: 849ms
770:	learn: 0.2985984	total: 2.85s	remaining: 847ms
771:	learn: 0.2985151	total: 2.85s	remaining: 843ms
772:	learn: 0.2983668	total: 2.86s	remaining: 839ms
773:	learn: 0.2983332	total: 2.86s	remaining: 835ms
774:	learn: 0.2982197	total: 2.86s	remaining: 831ms
775:	learn: 0.2981655	total: 2.87s	remaining: 828ms
776:	learn: 0.2979590	total: 2.88s	remaining: 826ms
777:	learn: 0.2979560	total: 2.88s	remaining: 822ms
778:	learn: 0.2978823	total: 2.88s	remaining: 818ms
779:	learn: 0.2977019	total: 2.89s	remaining: 814ms
780:	learn: 0.2974459	total: 2.89s	remaining: 810ms
781:	learn: 0.2972164	total: 2.89s	remaining: 806ms
782:	learn: 0.2970422	total: 2.89s	remaining: 802ms
783:	learn: 0.2968913	total: 2.9s	remaining: 800ms
784:	learn: 0.2968196	total: 2.9s	remaining: 796ms
785:	learn: 0.2966360	total: 2.91s	remaining: 792ms
786:	learn: 0.2964583	total: 2.91s	remaining: 788ms
787:	learn: 0.2963692	total: 2.91s	remaining: 784ms
788:	learn: 0.

929:	learn: 0.2824828	total: 3.53s	remaining: 266ms
930:	learn: 0.2823424	total: 3.54s	remaining: 263ms
931:	learn: 0.2822377	total: 3.54s	remaining: 259ms
932:	learn: 0.2822072	total: 3.55s	remaining: 255ms
933:	learn: 0.2821461	total: 3.55s	remaining: 251ms
934:	learn: 0.2820645	total: 3.55s	remaining: 247ms
935:	learn: 0.2819566	total: 3.56s	remaining: 243ms
936:	learn: 0.2817439	total: 3.56s	remaining: 239ms
937:	learn: 0.2816918	total: 3.57s	remaining: 236ms
938:	learn: 0.2816462	total: 3.57s	remaining: 232ms
939:	learn: 0.2816035	total: 3.58s	remaining: 229ms
940:	learn: 0.2815584	total: 3.58s	remaining: 225ms
941:	learn: 0.2814627	total: 3.58s	remaining: 221ms
942:	learn: 0.2813716	total: 3.59s	remaining: 217ms
943:	learn: 0.2813071	total: 3.59s	remaining: 213ms
944:	learn: 0.2812301	total: 3.6s	remaining: 209ms
945:	learn: 0.2810620	total: 3.6s	remaining: 205ms
946:	learn: 0.2810289	total: 3.6s	remaining: 202ms
947:	learn: 0.2809477	total: 3.6s	remaining: 198ms
948:	learn: 0.28

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,83.277217,81.027706,3.633825
2,Skl HistGBM,83.277217,81.027706,3.672802
3,CatBoost,83.277217,81.248202,50.023676
4,LightGBM,82.491582,80.886300,3.803831
5,XGBoost,81.930415,80.430927,6.128444
6,AdaBoost,81.144781,79.903653,3.472988
7,Extra Trees,81.032548,79.536957,4.570154
8,Random Forest,80.920314,79.390758,5.980313
9,Decision Tree,79.124579,77.823315,0.518630


In [73]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [77]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]

# Fit best model with all data

# YOUR CODE HERE
best_model = tree_classifiers['Skl GBM']
best_model.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(random_state=0))])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [81]:
x_test.isnull().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
Title        1
dtype: int64

In [79]:
# test_pred = # Get the predictions for x_test

# YOUR CODE HERE
test_preds = best_model.predict(x_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [ ]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

In [ ]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"